In [1]:
import os

import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from utils import tabularModel, titanicDataset

from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.suggest.bayesopt import BayesOptSearch

import time
%matplotlib inline

In [2]:
train_data = pd.read_csv(os.path.join('data', 'titanic', 'test.csv'))

In [3]:
train_data.head(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


In [4]:
def train(config):
    batch_size = int(config['batch_size'])
    num_embedding = int(config['num_embedding'])
    learning_rate = config['learning_rate']
    fc_size = (int(config['fc_size_0']), int(config['fc_size_1']))
    dropout = (config['dropout_0'], config['dropout_1'])
    dataset = titanicDataset(os.path.join('/ldap_home', 'chang.liao', '4483A3', 'data', 'titanic', 'train.csv'))
    train_dataset, valid_dataset = torch.utils.data.random_split(dataset,
                                                                 [int(0.8 * len(dataset)),
                                                                  len(dataset) - int(0.8 * len(dataset))])
    train_dataloader = DataLoader(train_dataset,
                                  batch_size=batch_size, shuffle=True, num_workers=4)
    valid_dataloader = DataLoader(valid_dataset,
                                 batch_size=len(valid_dataset), shuffle=False, num_workers=4)
    categorical_columns = ['Pclass', 'Sex', 'Embarked', 'binned_family']
    categories = [3, 2, 3, 4]
    continuous_columns = ['Fare', 'Age']
    embedding_size = list(zip(categories, [num_embedding] * 4))
    model = tabularModel(embedding_size, categorical_columns, continuous_columns, fc_size, dropout).double()
    optim = torch.optim.Adagrad(model.parameters(), lr=learning_rate, weight_decay=0.95)
    lossfunc = nn.CrossEntropyLoss()
    num_epoch = 300
    best_accu = 0
    cnt = 0
    model.train()
    for epoch in range(num_epoch):
        for i, (x, y) in enumerate(train_dataloader):
            optim.zero_grad()
            pred = model(x)
            loss = lossfunc(pred, y.squeeze().long())
            loss.backward()
            optim.step()

            if i % 10 == 0 or i == (len(train_dataset) / batch_size) - 1 :
                with torch.no_grad():
                    model.eval()
                    x, y = next(iter(valid_dataloader))
                    pred = model(x)
                    valid_loss = lossfunc(pred, y.squeeze().long())
                    accu = (pred.argmax(-1) == y.squeeze()).sum().item() / len(valid_dataset)
                # print("Epoch: {} | Train loss: {:.4f} | Valid loss: {:.4f} | Valid Accu: {:.4f}".format(
                #     epoch, loss.item(), valid_loss.item(), accu
                # ))
                model.train()
        # print("\n {:.4f} {:.4f} {} \n".format(1.02 * best_accu, accu, cnt))
        if accu >= best_accu:
            cnt = 0
            best_accu = accu
            # torch.save(model.state_dict(), os.path.join('model', 'torch_nn',
            #                                             'model-{}.pt'.format(epoch)))
        else:
            cnt += 1

        if cnt == 50:
            # print("Early stopped!")
            break
    return best_accu

In [5]:
def raytune_trainable(config, checkpoint_dir=None):
    tune.report(accu=train(config))

In [6]:
search_space = {
    'batch_size': (2, 64),
    'num_embedding': (4, 256),
    'learning_rate': (1e-4, 0.1),
    'fc_size_0': (64, 256),
    'fc_size_1': (8, 128),
    'dropout_0': (0.1, 0.8),
    'dropout_1': (0.1, 0.8)
}

In [7]:
algo = BayesOptSearch(space=search_space, metric='accu', mode='max', random_search_steps=20)
start = time.time()
analysis = tune.run(raytune_trainable, search_alg=algo, num_samples=500, 
                    # checkpoint_freq=4,
                    # checkpoint_at_end=True,
                    local_dir=os.curdir,
                    resources_per_trial={'cpu': 18},
                    scheduler = ASHAScheduler(
                        metric='accu',
                        mode='max'),
                    raise_on_failed_trial=False,
                    reuse_actors=True,
                    queue_trials=True
                   )
end = time.time()
print(end-start)

2020-11-11 00:46:42,053	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265
2020-11-11 00:46:44,237	WARNING tune.py:401 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.
2020-11-11 00:46:44,244	INFO logger.py:201 -- pip install 'ray[tune]' to see TensorBoard files.
2020-11-11 00:46:44,245	WARNING logger.py:343 -- Could not instantiate TBXLogger: No module named 'tensorboardX'.


Trial name,status,loc,batch_size,dropout_0,dropout_1,fc_size_0,fc_size_1,learning_rate,num_embedding
raytune_trainable_50b8c084,RUNNING,,25.2215,0.7655,0.612396,178.942,26.7222,0.0156839,18.6371


Result for raytune_trainable_50c0d24c:
  accu: 0.8435754189944135
  date: 2020-11-11_00-47-08
  done: false
  experiment_id: bf991e42b45b4916836e549cf769f086
  experiment_tag: 6_batch_size=52.121,dropout_0=0.31323,dropout_1=0.16837,fc_size_0=195.37,fc_size_1=60.818,learning_rate=0.012292,num_embedding=128.78
  hostname: shopee-ds-ip-10-128-148-194.idata-server.shopee.io
  iterations_since_restore: 1
  node_ip: 10.128.148.194
  pid: 55918
  time_since_restore: 23.661699771881104
  time_this_iter_s: 23.661699771881104
  time_total_s: 23.661699771881104
  timestamp: 1605026828
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 50c0d24c
  


Trial name,status,loc,batch_size,dropout_0,dropout_1,fc_size_0,fc_size_1,learning_rate,num_embedding,iter,total time (s),accu
raytune_trainable_50b8c084,RUNNING,,25.2215,0.7655,0.612396,178.942,26.7222,0.0156839,18.6371,,,
raytune_trainable_50bc8bec,RUNNING,,55.7029,0.520781,0.595651,67.9522,124.389,0.083261,57.5095,,,
raytune_trainable_50bd96a4,RUNNING,,13.2731,0.228383,0.31297,164.753,59.8334,0.0291938,158.187,,,
raytune_trainable_50bea36e,RUNNING,,10.6486,0.304501,0.356453,151.565,102.221,0.0200474,133.587,,,
raytune_trainable_50bfe7b0,RUNNING,,38.7297,0.132515,0.525281,96.7406,15.8062,0.0948937,247.339,,,
raytune_trainable_50c0d24c,RUNNING,10.128.148.194:55918,52.1206,0.31323,0.16837,195.373,60.8183,0.0122916,128.785,1,23.6617,0.843575
raytune_trainable_50c1d020,RUNNING,,4.13209,0.736524,0.281146,191.204,45.4053,0.0520548,141.771,,,
raytune_trainable_50c2c19c,PENDING,,13.461,0.778709,0.642593,244.384,115.379,0.0598302,236.312,,,


Result for raytune_trainable_50b8c084:
  accu: 0.8491620111731844
  date: 2020-11-11_00-47-09
  done: false
  experiment_id: 096e855f4aa14acbb1d7549aebc93275
  experiment_tag: 1_batch_size=25.221,dropout_0=0.7655,dropout_1=0.6124,fc_size_0=178.94,fc_size_1=26.722,learning_rate=0.015684,num_embedding=18.637
  hostname: shopee-ds-ip-10-128-148-194.idata-server.shopee.io
  iterations_since_restore: 1
  node_ip: 10.128.148.194
  pid: 55978
  time_since_restore: 24.779476642608643
  time_this_iter_s: 24.779476642608643
  time_total_s: 24.779476642608643
  timestamp: 1605026829
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 50b8c084
  
Result for raytune_trainable_50bd96a4:
  accu: 0.8156424581005587
  date: 2020-11-11_00-47-36
  done: true
  experiment_id: 19c12ffb49da441c8a1567d35382bb47
  experiment_tag: 3_batch_size=13.273,dropout_0=0.22838,dropout_1=0.31297,fc_size_0=164.75,fc_size_1=59.833,learning_rate=0.029194,num_embedding=158.19
  hostname: shopee-ds-ip-10-128-148

...

Trial name,status,loc,batch_size,dropout_0,dropout_1,fc_size_0,fc_size_1,learning_rate,num_embedding,iter,total time (s),accu
raytune_trainable_50b8c084,TERMINATED,,25.2215,0.7655,0.612396,178.942,26.7222,0.0156839,18.6371,1,24.7795,0.849162
raytune_trainable_50bc8bec,TERMINATED,,55.7029,0.520781,0.595651,67.9522,124.389,0.083261,57.5095,1,141.278,0.620112
raytune_trainable_50bd96a4,TERMINATED,,13.2731,0.228383,0.31297,164.753,59.8334,0.0291938,158.187,1,51.4029,0.815642
raytune_trainable_50bea36e,TERMINATED,,10.6486,0.304501,0.356453,151.565,102.221,0.0200474,133.587,1,63.0587,0.798883
raytune_trainable_50bfe7b0,TERMINATED,,38.7297,0.132515,0.525281,96.7406,15.8062,0.0948937,247.339,1,222.677,0.648045
raytune_trainable_50c0d24c,TERMINATED,,52.1206,0.31323,0.16837,195.373,60.8183,0.0122916,128.785,1,23.6617,0.843575
raytune_trainable_50c1d020,TERMINATED,,4.13209,0.736524,0.281146,191.204,45.4053,0.0520548,141.771,1,1734.72,0.608939
raytune_trainable_50c2c19c,TERMINATED,,13.461,0.778709,0.642593,244.384,115.379,0.0598302,236.312,1,79.2932,0.642458
raytune_trainable_5f54d90c,TERMINATED,,7.48654,0.237188,0.131659,126.463,54.6413,0.0272078,212.842,1,88.2683,0.765363
raytune_trainable_5ffe6062,TERMINATED,,24.1187,0.296654,0.479887,91.0575,104.264,0.00754761,252.696,1,174.141,0.77095


2020-11-11 01:15:39,902	INFO trial_runner.py:1019 -- Blocking for next trial...
2020-11-11 01:15:40,903	INFO trial_runner.py:1019 -- Blocking for next trial...
2020-11-11 01:15:41,906	INFO trial_runner.py:1019 -- Blocking for next trial...
2020-11-11 01:15:42,908	INFO trial_runner.py:1019 -- Blocking for next trial...
2020-11-11 01:15:43,910	INFO trial_runner.py:1019 -- Blocking for next trial...
2020-11-11 01:15:44,912	INFO trial_runner.py:1019 -- Blocking for next trial...
2020-11-11 01:15:45,914	INFO trial_runner.py:1019 -- Blocking for next trial...
2020-11-11 01:15:46,916	INFO trial_runner.py:1019 -- Blocking for next trial...
2020-11-11 01:15:47,918	INFO trial_runner.py:1019 -- Blocking for next trial...
2020-11-11 01:15:48,920	INFO trial_runner.py:1019 -- Blocking for next trial...
2020-11-11 01:15:49,922	INFO trial_runner.py:1019 -- Blocking for next trial...
2020-11-11 01:15:50,924	INFO trial_runner.py:1019 -- Blocking for next trial...
2020-11-11 01:15:51,926	INFO trial_runne

Trial name,status,loc,batch_size,dropout_0,dropout_1,fc_size_0,fc_size_1,learning_rate,num_embedding,iter,total time (s),accu
raytune_trainable_50b8c084,TERMINATED,,25.2215,0.7655,0.612396,178.942,26.7222,0.0156839,18.6371,1,24.7795,0.849162
raytune_trainable_50bc8bec,TERMINATED,,55.7029,0.520781,0.595651,67.9522,124.389,0.083261,57.5095,1,141.278,0.620112
raytune_trainable_50bd96a4,TERMINATED,,13.2731,0.228383,0.31297,164.753,59.8334,0.0291938,158.187,1,51.4029,0.815642
raytune_trainable_50bea36e,TERMINATED,,10.6486,0.304501,0.356453,151.565,102.221,0.0200474,133.587,1,63.0587,0.798883
raytune_trainable_50bfe7b0,TERMINATED,,38.7297,0.132515,0.525281,96.7406,15.8062,0.0948937,247.339,1,222.677,0.648045
raytune_trainable_50c0d24c,TERMINATED,,52.1206,0.31323,0.16837,195.373,60.8183,0.0122916,128.785,1,23.6617,0.843575
raytune_trainable_50c1d020,TERMINATED,,4.13209,0.736524,0.281146,191.204,45.4053,0.0520548,141.771,1,1734.72,0.608939
raytune_trainable_50c2c19c,TERMINATED,,13.461,0.778709,0.642593,244.384,115.379,0.0598302,236.312,1,79.2932,0.642458
raytune_trainable_5f54d90c,TERMINATED,,7.48654,0.237188,0.131659,126.463,54.6413,0.0272078,212.842,1,88.2683,0.765363
raytune_trainable_5ffe6062,TERMINATED,,24.1187,0.296654,0.479887,91.0575,104.264,0.00754761,252.696,1,174.141,0.77095


2020-11-11 01:25:40,160	INFO trial_runner.py:1019 -- Blocking for next trial...
2020-11-11 01:25:41,162	INFO trial_runner.py:1019 -- Blocking for next trial...
2020-11-11 01:25:42,164	INFO trial_runner.py:1019 -- Blocking for next trial...
2020-11-11 01:25:43,166	INFO trial_runner.py:1019 -- Blocking for next trial...
2020-11-11 01:25:44,167	INFO trial_runner.py:1019 -- Blocking for next trial...
2020-11-11 01:25:45,169	INFO trial_runner.py:1019 -- Blocking for next trial...
2020-11-11 01:25:46,172	INFO trial_runner.py:1019 -- Blocking for next trial...
2020-11-11 01:25:47,174	INFO trial_runner.py:1019 -- Blocking for next trial...
2020-11-11 01:25:48,176	INFO trial_runner.py:1019 -- Blocking for next trial...
2020-11-11 01:25:49,178	INFO trial_runner.py:1019 -- Blocking for next trial...
2020-11-11 01:25:50,180	INFO trial_runner.py:1019 -- Blocking for next trial...
2020-11-11 01:25:51,182	INFO trial_runner.py:1019 -- Blocking for next trial...
2020-11-11 01:25:52,184	INFO trial_runne

KeyboardInterrupt: 

In [9]:
analysis.best_result

NameError: name 'analysis' is not defined